In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24235
2,Huelva,Confirmados PDIA 14 días,1413
3,Huelva,Tasa PDIA 14 días,"275,34735078044315"
4,Huelva,Confirmados PDIA 7 días,464
5,Huelva,Total Confirmados,24465
6,Huelva,Curados,13795
7,Huelva,Fallecidos,266


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24235.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6469.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 999 personas en los últimos 7 días 

Un positivo PCR cada 332 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24235.0,464.0,1413.0,513170.0,90.418380,275.347351,49.0
Huelva-Costa,14781.0,292.0,919.0,289548.0,100.846837,317.391244,30.0
Condado-Campiña,6701.0,142.0,383.0,156231.0,90.891052,245.149810,16.0
Huelva (capital),6469.0,144.0,433.0,143837.0,100.113323,301.035200,15.0
Almonte,819.0,44.0,99.0,24507.0,179.540539,403.966214,9.0
Bollullos Par del Condado,583.0,24.0,56.0,14387.0,166.817266,389.240286,7.0
Punta Umbría,646.0,14.0,43.0,15355.0,91.175513,280.039075,5.0
Cartaya,905.0,43.0,64.0,20083.0,214.111438,318.677488,5.0
Lepe,1961.0,38.0,123.0,27880.0,136.298422,441.176471,4.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,164.0,1.0,26.0,2885.0,34.662045,901.213172,1.0
Cumbres de San Bartolomé,5.0,2.0,2.0,376.0,531.914894,531.914894,0.0
Isla Cristina,1979.0,24.0,100.0,21393.0,112.186229,467.442621,0.0
Lepe,1961.0,38.0,123.0,27880.0,136.298422,441.176471,4.0
Almonte,819.0,44.0,99.0,24507.0,179.540539,403.966214,9.0
Bollullos Par del Condado,583.0,24.0,56.0,14387.0,166.817266,389.240286,7.0
Ayamonte,1174.0,11.0,74.0,21104.0,52.122820,350.644428,1.0
Almonaster la Real,54.0,1.0,6.0,1788.0,55.928412,335.570470,0.0
Cartaya,905.0,43.0,64.0,20083.0,214.111438,318.677488,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villablanca,164.0,1.0,26.0,2885.0,34.662045,901.213172,1.0,0.038462
Bonares,177.0,1.0,13.0,6060.0,16.501650,214.521452,-1.0,0.076923
Palos de la Frontera,435.0,3.0,31.0,11742.0,25.549310,264.009538,-3.0,0.096774
Aracena,456.0,1.0,10.0,8255.0,12.113870,121.138704,0.0,0.100000
Ayamonte,1174.0,11.0,74.0,21104.0,52.122820,350.644428,1.0,0.148649
Almonaster la Real,54.0,1.0,6.0,1788.0,55.928412,335.570470,0.0,0.166667
Puebla de Guzmán,81.0,1.0,6.0,3092.0,32.341527,194.049159,0.0,0.166667
Palma del Condado (La),961.0,6.0,26.0,10801.0,55.550412,240.718452,-1.0,0.230769
Isla Cristina,1979.0,24.0,100.0,21393.0,112.186229,467.442621,0.0,0.240000
